<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Read-a-piece-of-news-for-example" data-toc-modified-id="Read-a-piece-of-news-for-example-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Read a piece of news for example</a></span></li><li><span><a href="#查找说的近义词" data-toc-modified-id="查找说的近义词-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>查找说的近义词</a></span></li><li><span><a href="#NER---extract-all-the-names-from-the-content" data-toc-modified-id="NER---extract-all-the-names-from-the-content-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>NER - extract all the names from the content</a></span></li><li><span><a href="#依存句法分析" data-toc-modified-id="依存句法分析-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>依存句法分析</a></span></li><li><span><a href="#Extract-from-name" data-toc-modified-id="Extract-from-name-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Extract from name</a></span></li><li><span><a href="#Extract-from-'say'-words" data-toc-modified-id="Extract-from-'say'-words-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Extract from 'say' words</a></span></li></ul></div>

[Reference1 - other solutions](https://github.com/enix223/nlp-course/blob/master/notebooks/news_options.ipynb)

[Reference2 - pyltp](https://pyltp.readthedocs.io/zh_CN/latest/api.html#id13)

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import jieba
import pyltp
import pandas as pd
from gensim.models.word2vec import Word2Vec

In [4]:
LTP_DATA_DIR = 'mydata/ltp_data_v3.4.0'  # ltp模型目录的路径
cws_model_path = os.path.join(LTP_DATA_DIR, 'cws.model')
pos_model_path = os.path.join(LTP_DATA_DIR, 'pos.model')
ner_model_path = os.path.join(LTP_DATA_DIR, 'ner.model')
par_model_path = os.path.join(LTP_DATA_DIR, 'parser.model')
srl_model_path = os.path.join(LTP_DATA_DIR, 'pisrl.model')

In [6]:
segmentor = pyltp.Segmentor()
segmentor.load(cws_model_path)

postagger = pyltp.Postagger()
postagger.load(pos_model_path)

recognizer = pyltp.NamedEntityRecognizer()
recognizer.load(ner_model_path)

parser = pyltp.Parser()
parser.load(par_model_path)

labeller = pyltp.SementicRoleLabeller() # 初始化实例
labeller.load(srl_model_path)  # 加载模型

In [8]:
# news path
NEWS_DATA_PATH = 'mydata/sqlResult_1558435.csv'
df = pd.read_csv(NEWS_DATA_PATH, encoding='gb18030')

> 已通过Word2Vec Model和Wiki Data 将“说”的近义词找到，存放于本地，现在读取

In [35]:
say_similar = pd.read_csv("similar_said.csv", encoding = "gbk")
said_list = list(say_similar['similar_of_said'])

# Read a piece of news for example

In [36]:
contents = df[~df['content'].isnull()]['content']

In [37]:
news = contents[96]

In [38]:
news

'中新网6月23日电 (记者潘旭临) 意大利航空首席商务官乔治先生22日在北京接受中新网记者专访时表示，意航确信中国市场对意航的重要性，目前意航已将发展中国市场提升到战略层级的高度，未来，意航将加大在华布局，提升业务水平。\r\n到意大利航空履职仅7个月的乔治，主要负责包括中国市场在内的亚太业务。此次北京之行，目标是加深对中国市场的了解，寻找合作和投资良机。\r\n他说，“自己是第一次访问北京，也是第一次到中国访问。虽然此次时间短暂，但中国传统文化、古代建筑以及可口的中餐等，都给他留下非常美好的印象。”\r\n自从去年7月，意大利航空首次开启罗马至北京往返直飞航线后，目前每周有4个航班，提供近千个座位，业绩状况一直稳步有升。乔治称，随着对华业务不断提升，意航明年可能会将每周4班提高到每天一班。同时，意航会借罗马新航站楼启用之际，吸引更多中国旅客到意大利旅游和转机。此外，还将加大对北京直飞航线的投资，如翻新航班座椅，增加电视中有关中国内容的娱乐节目、提高机上中文服务、餐饮服务、完善意航中文官方网站，提升商务舱和普通舱的舒适度等。\r\n他表示，随着中国经济持续稳步增长，中国将很快跃居成为世界第一出境旅游大国。去年中国出境旅游突破了一亿人次，来意大利旅游的中国人也超过百万人次，今年还会增加。在面对如此巨大的旅游市场时，作为本土老牌航空公司，意航只有练好“内功”，不断完善和提高自身业务能力，才能在市场竞争中，占得先机，立于不败之地。\r\n在谈到目前意大利航空的经营状况时，乔治先生坦言，走过70年的意大利航空，目前正在经历如裁员、管理层重组等历史上又一次转型期。 乔治表示，意航转型期也是投资者的机遇期，意大利航空已做好吸引全球业界投资的准备，以迅速扩充实力，目前已有33家国际投资机构向意航提出投资申请，能否接受、如何接受，意航需做全面评估才会决定。\r\n乔治表示，在转型期内，意航业务如常，资金充分，并计划在下半年和明年增加中远程航线的投资，尤其是稳固和加强像中国这样的大市场投资。\r\n'

In [39]:
words = segmentor.segment(news.replace('\r\n', ''))

In [40]:
postags = postagger.postag(words)

In [41]:
netags = recognizer.recognize(words, postags)

In [42]:
pd.DataFrame({"words":(" ".join(words).split()), "postags":" ".join(postags).split(), "netags":" ".join(netags).split()})

,netags,postags,words
0,O,n,中新网
1,O,nt,6月
2,O,nt,23日
3,O,n,电
4,O,wp,(
5,O,n,记者
6,S-Nh,nh,潘旭临
7,O,wp,)
8,B-Ni,ns,意大利
9,I-Ni,n,航空


# 查找说的近义词

In [43]:
say_positions = [(w, i) for i, w in enumerate(words) if w in said_list]

In [44]:
say_positions

[('表示', 22),
 ('确信', 25),
 ('说', 103),
 ('称', 188),
 ('表示', 284),
 ('坦言', 386),
 ('表示', 412),
 ('表示', 468)]

# NER - extract all the names from the content

In [45]:
all_names_positions = [(i, tag) for i, tag in enumerate(netags) if 'Nh' in tag]

In [46]:
all_names_positions

[(6, 'S-Nh'),
 (12, 'S-Nh'),
 (69, 'S-Nh'),
 (187, 'S-Nh'),
 (384, 'S-Nh'),
 (411, 'S-Nh'),
 (467, 'S-Nh')]

In [47]:
all_names = [(words[name[0]], name[0]) for name in all_names_positions]

In [48]:
all_names

[('潘旭临', 6),
 ('乔治', 12),
 ('乔治', 69),
 ('乔治', 187),
 ('乔治', 384),
 ('乔治', 411),
 ('乔治', 467)]

In [49]:
names = set(map(lambda x: x[0], all_names))

In [50]:
names

{'乔治', '潘旭临'}

# 依存句法分析

In [51]:
arcs = parser.parse(words, postags)

In [60]:
# print("\n".join("%d:%s=>%d:%s" % 
#         (i, '********{}********'.format(words[i]) if words[i] in said_list else words[i], arc.head, arc.relation) 
#                 for i, arc in enumerate(arcs)))

# Extract from name

In [53]:
def get_all_words_related_to(index):
    return [words[i] for i, arc in enumerate(arcs) if arc.head == index]

In [54]:
candidates = [(name, arcs[name[1]].relation, arcs[name[1]].head) 
              for name in all_names if arcs[name[1]].relation == 'SBV']

In [55]:
candidates

[(('乔治', 187), 'SBV', 189),
 (('乔治', 411), 'SBV', 413),
 (('乔治', 467), 'SBV', 469)]

In [56]:
def is_sentence_end(w):
    return w in ["。", "!", "！"]

In [57]:
for candidate in candidates:
    sentence = ''
    for w in words[candidate[2] + 1:]:
        sentence += w
        if is_sentence_end(w):
            print('{}:{}'.format(candidate[0][0], sentence))
            break

乔治:随着对华业务不断提升，意航明年可能会将每周4班提高到每天一班。
乔治:意航转型期也是投资者的机遇期，意大利航空已做好吸引全球业界投资的准备，以迅速扩充实力，目前已有33家国际投资机构向意航提出投资申请，能否接受、如何接受，意航需做全面评估才会决定。
乔治:在转型期内，意航业务如常，资金充分，并计划在下半年和明年增加中远程航线的投资，尤其是稳固和加强像中国这样的大市场投资。


# Extract from 'say' words

In [58]:
options = []
for say_word, pos in say_positions:
    option = {'say': say_word}
    # Get the name who say the words
    for i in range(pos, 0, -1):
        w = words[i]
        if w in names:
            option['name'] = w
            break
    
    sentence = ''
    for w in words[pos + 1:]:
        sentence += w
        if is_sentence_end(w):
            option['sentence'] = sentence
            break
    
    options.append(option)

In [59]:
for option in options:
    print('{} {} {}'.format(option['name'], option['say'], option['sentence']))

乔治 表示 ，意航确信中国市场对意航的重要性，目前意航已将发展中国市场提升到战略层级的高度，未来，意航将加大在华布局，提升业务水平。
乔治 确信 中国市场对意航的重要性，目前意航已将发展中国市场提升到战略层级的高度，未来，意航将加大在华布局，提升业务水平。
乔治 说 ，“自己是第一次访问北京，也是第一次到中国访问。
乔治 称 ，随着对华业务不断提升，意航明年可能会将每周4班提高到每天一班。
乔治 表示 ，随着中国经济持续稳步增长，中国将很快跃居成为世界第一出境旅游大国。
乔治 坦言 ，走过70年的意大利航空，目前正在经历如裁员、管理层重组等历史上又一次转型期。
乔治 表示 ，意航转型期也是投资者的机遇期，意大利航空已做好吸引全球业界投资的准备，以迅速扩充实力，目前已有33家国际投资机构向意航提出投资申请，能否接受、如何接受，意航需做全面评估才会决定。
乔治 表示 ，在转型期内，意航业务如常，资金充分，并计划在下半年和明年增加中远程航线的投资，尤其是稳固和加强像中国这样的大市场投资。
